In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_line, get_post_df
import altair as alt

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

##

In [2]:
log_dir = './log/20201224_122429_pseudo//'
ps = load_obj('mcmc_post_samples', log_dir)

## Post process loadings for sign flips

In [3]:
nsim = ps['beta'].shape[0]
nrows = ps['beta'].shape[1]
for n in range(nsim):
    for i in range(nrows):
        sign = np.sign(ps['beta'][n,0,0])
        ps['beta'][n,1:] = sign * ps['beta'][n,1:]

## Add a chain dimension to generalize

In [4]:
ps['beta'] = ps['beta'].reshape((ps['beta'].shape[0],1,ps['beta'].shape[1],ps['beta'].shape[2]))

In [5]:
ps['alpha'] = ps['alpha'].reshape((ps['alpha'].shape[0],1,ps['alpha'].shape[1]))

## Plot MCMC samples

In [6]:
param = 'beta'
df = get_post_df(ps[param]) 
df['source'] = 'mcmc'
plot_line(df, height=100)

alt.Chart(...)

In [7]:
param = 'alpha'
df = get_post_df(ps[param]) 
df['source'] = 'mcmc'
plot_density(df, height=100)

alt.Chart(...)

## Plot quantiles

In [8]:
param = 'beta'
df = get_post_df(ps[param])
df_quant = df.groupby(['chain', 'row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['chain', 'row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['chain', 'row', 'col'])

# simple quantile chart
df['source'] = 'mcmc'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

## chart overlaying quantiles with real data

In [9]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'mcmc'
df

,index,q1,q2,source
0,r_0.c_0,-1.762009,0.305344,mcmc
1,r_1.c_0,-1.454091,0.764397,mcmc
2,r_2.c_0,-1.432106,0.495861,mcmc
3,r_3.c_0,-1.774923,0.161178,mcmc
4,r_4.c_0,-1.914249,0.490481,mcmc
5,r_5.c_0,-1.458117,0.623694,mcmc


In [10]:
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Color('source')
)
c1.facet(
       'index',
    columns=2
    )

    

alt.FacetChart(...)

In [11]:
data = load_obj('data', log_dir)

In [12]:
dd = pd.DataFrame(-data.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
dd

,index,data
0,r_0.c_0,-1.0
1,r_1.c_0,-0.7
2,r_2.c_0,-0.8
3,r_3.c_0,-0.5
4,r_4.c_0,-0.9
5,r_5.c_0,-0.6


In [13]:
plot_data = df.merge(dd, on=['index'])

In [14]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=2
    )


alt.FacetChart(...)